In [ ]:
import numpy as np
from sympy import *
from sympy.abc import x, y
import matplotlib.pyplot as plt
from scipy import linalg

def get_m0(n: int):
    """
    get a fundamental matrix in statistics, "centering matrix" that is used to transform data to
    deviations from their mean
    :param n: length of data
    :return: $M_0$
    """
    return (-1 / n) * np.ones((n, n)) + np.eye(n)


def sum(array):
    n = array.shape[0]
    i = np.ones(n)

    return np.dot(i, array)

def mean(array):
    import numpy as np
    n = array.shape[0]
    i = np.ones(n)
    return (1 / n) * np.dot(i, array)


def mean_deviation(array):
    n = array.shape[0]
    m0 = get_m0(n)
    return np.matmul(m0, array)


def sum_of_squares(array):
    return np.sum(array ** 2)


def sum_of_squared_deviation(array):
    return sum_of_squares(mean_deviation(array))


def sum_of_squres_matrix(a, b):
    z = np.c_[a, b]
    return z.T.matmul(get_m0(z.shape[0])).matmul(z)


def rref(matrix):
    m = Matrix(matrix)
    return np.array(m.rref()[0].tolist()).astype(np.float64)

def rank(matrix):
    return np.linalg.matrix_ranK(matrix)

def det(matrix):
    return np.linalg.det(matrix)

def inv(matrix):
    return np.linalg.inv(matrix)

def l1_norm(matrix):
    return np.abs(np.array(matrix)).sum()

def l2_norm(matrix):
    return np.sqrt((np.array(matrix) ** 2).sum())

def dot(arr1, arr2):
    return np.dot(np.array(arr1), np.array(arr2))

def arccos(angle):
    return np.arccos(angle)

def arccos_deg(angle):
    return arccos(angle) * 180 / np.pi

def orthogonal(arr1, arr2):
    return dot(arr1, arr2) == 0

def normalize(v):
    return v/l2_norm(v)

def inner_product(x, y, A):
    x_arr = np.array(x)
    y_arr = np.array(y)
    A_arr = np.array(A)
    return np.matmul(np.matmul(x_arr, A_arr), y_arr.T)
def proj_matrix(U):
    U_arr = np.array(U)

    return U_arr.dot(inv(U_arr.T.dot(U_arr))).dot(U_arr.T)

def proj(v, U):
    v_arr = np.array(v)

    return proj_matrix(U).dot(v_arr)

def gram_schmidt(B):
    B_arr = np.array(B)
    for i in range(B_arr.shape[1]):
        if i == 0:
            U = normalize(B_arr[:, i])
            U = U.reshape(U.shape[0], -1)
        else:
            U_j = normalize(B_arr[:, i] - proj(B_arr[:, i], U))
            U = np.c_[U, U_j]
    return U

def rotate(x, deg):
    x_arr = np.array(x)
    rad = deg*np.pi/180
    return np.matmul(np.array([[np.cos(rad), -np.sin(rad)], [np.sin(rad), np.cos(rad)]]), x_arr)

def det_laplace_expansion(A, checked=False):
    arr = np.array(A)
    if not checked:
        if len(arr.shape) == 1:
            assert arr.shape[0] == 1, "Input matrix is not square."
        else:
            assert arr.shape[0] == arr.shape[1], "Input matrix is not square."
    check = True
    if (arr.shape[0] == 1):
        return arr
    else:
        det = 0
        for i in range(arr.shape[0]):
            row_index = np.array([x for x in range(arr.shape[0]) if x != i])[:,np.newaxis]
            print(arr.shape[0])
            print(row_index)
            column_index = np.array([x for x in range(arr.shape[1]) if x != 0])
            print(column_index)
            print(arr[row_index, column_index])
            det += ((-1)**i) * det_laplace_expansion(arr[row_index, column_index], checked)
        return det

def trace(A):
    arr = np.array(A)
    return np.trace(A)

def charpoly(A):
    M = Matrix(A)
    return M.charpoly(x).as_expr()

def eig(A):
    M = np.array(A)
    return np.linalg.eig(M)

def cholesky(lA):
    arrA = np.array(lA)
    return np.linalg.cholesky(arrA)


def plotVectors(vecs, cols, alpha=1):
    """
    Plot set of vectors.

    Parameters
    ----------
    vecs : array-like
        Coordinates of the vectors to plot. Each vectors is in an array. For
        instance: [[1, 3], [2, 2]] can be used to plot 2 vectors.
    cols : array-like
        Colors of the vectors. For instance: ['red', 'blue'] will display the
        first vector in red and the second in blue.
    alpha : float
        Opacity of vectors

    Returns:

    fig : instance of matplotlib.figure.Figure
        The figure of the vectors
    """
    plt.figure()
    plt.axvline(x=0, color='#A9A9A9', zorder=0)
    plt.axhline(y=0, color='#A9A9A9', zorder=0)

    for i in range(len(vecs)):
        x = np.concatenate([[0,0],vecs[i]])
        plt.quiver([x[0]],
                   [x[1]],
                   [x[2]],
                   [x[3]],
                   angles='xy', scale_units='xy', scale=1, color=cols[i],
                   alpha=alpha)

def eigen_decomposition(lA):
    arrA = np.array(lA)
    results = eig(arrA)
    assert det(results[1]) != 0, "Defective matrix cannot be diagonalized."
    return (results[1], np.diag(results[0]), inv(results[1]))

def svd(lA):
    arrA = np.array(lA)
    U, S, VT = linalg.svd(arrA)
    Sigma = np.zeros(arrA.shape)
    k = min(arrA.shape)
    Sigma[:k, :k] = np.diag(S)
    return U, Sigma, VT

def pinv(lA):
    arrA = np.array(A)
    U, s, VT = np.linalg.svd(A)
    d = 1.0/s
    D = np.zeros(A.shape)
    k = min(A.shape)
    D[:k, :k] = np.diag(d)
    return (VT.T).dot((D.T).dot(U.T))

def matrix_approximate(lA, n_components):
    arrA = np.array(lA)
    U, Sigma, VT = svd(arrA)
    Sigmak = Sigma[:, :n_components]
    VTk = VT[:n_components, :]
    print(U.shape, Sigmak.shape, VTk.shape)
    return U.dot(Sigmak.dot(VTk))

class LinearRegression:

    def __init__(self):
        self._params_ = None
        self._intercept_ = None
        self._coef_ = None

    @property
    def params_(self):
        return self._params_

    @params_.setter
    def params_(self, params):
        import numpy as np
        self._params_ = np.array(params)

    @property
    def intercept_(self):
        return self._intercept_

    @intercept_.setter
    def intercept_(self, intercept):
        import numpy as np
        self._intercept_ = np.array(intercept)

    @property
    def coef_(self):
        return self._coef_

    @coef_.setter
    def coef_(self, coef):
        import numpy as np
        self._coef_ = np.array(coef)

    def fit(self, X, y):
        n = X.shape[0]
        ones = np.ones(n)
        feat = np.c_[X, ones]
        inv = np.linalg.inv(np.matmul(feat.T, feat))
        self.params_ = np.matmul(np.matmul(inv, feat.T), y)
        self.coef_ = self.params_[0]
        self.intercept_ = self.params_[1]
        print(self.params_)

    def predict(self, x):
        n = x.shape[0]
        feat = np.c_[x, np.ones(n)]
        return np.matmul(feat, self.params_)

# 4.1

In [ ]:
A = [[1, 3, 5],
     [2, 4, 6],
     [0, 2, 4]]
det(A)

0.0

# 4.2

In [ ]:
A = [[2, 0, 1, 2, 0],
     [2, -1, 0, 1, 1,],
     [0, 1, 2, 1, 2],
     [-2 ,0, 2, -1, 2],
     [2, 0, 0, 1, 1]]
det(A)

6.000000000000003

# 4.3

## a

In [ ]:
lA = [[1, 0], [1, 1]]
eig(lA)

(array([1., 1.]), array([[ 0.00000000e+00,  2.22044605e-16],
        [ 1.00000000e+00, -1.00000000e+00]]))

In [ ]:
charpoly(lA)

x**2 - 2*x + 1

## b

In [ ]:
lB = [[-2, 2],
      [2, 1]]
eig(lB)

(array([-3.,  2.]), array([[-0.89442719, -0.4472136 ],
        [ 0.4472136 , -0.89442719]]))

# 4.4

In [ ]:
lA = [[0, -1, 1, 1],
      [-1, 1, -2, 3],
      [2, -1, 0, 0],
      [1, -1, 1, 0]]
eig(lA)[0]

array([ 2.        ,  1.        , -1.00000002, -0.99999998])

In [ ]:
eig(lA)[1]

array([[ 5.77350269e-01,  5.00000000e-01,  1.42763450e-08,
         1.42763441e-08],
       [ 1.70821396e-16,  5.00000000e-01, -7.07106774e-01,
         7.07106788e-01],
       [ 5.77350269e-01,  5.00000000e-01, -7.07106788e-01,
         7.07106774e-01],
       [ 5.77350269e-01,  5.00000000e-01,  2.56395030e-16,
        -2.30558502e-16]])

# 4.5

In [ ]:
lA = [[1, 0], [0, 1]]
lB = [[1, 0], [0, 0]]
lC = [[1, 1], [0, 1]]
lD = [[0, 1], [0, 0]]

A is diagonalizable and invertible.
B is singular but diagonalizable

In [ ]:
eigen_decomposition(lB)

(array([[1., 0.],
        [0., 1.]]), array([[1., 0.],
        [0., 0.]]), array([[1., 0.],
        [0., 1.]]))

C is invertible but not diagonalizable.

D is neither invertible and diagonalizable.

# 4.6

## a

A is not diagonalizable if $A \in \mathbb{R}^{3\times 3}$

In [ ]:
lA = [[2, 3, 0],
     [1, 4, 3],
     [0, 0, 1]]
charpoly(lA)

x**3 - 7*x**2 + 11*x - 5

In [ ]:
cp = np.polynomial.polynomial.Polynomial([-5, 11, -7, 3])

In [ ]:
print(cp)

poly([-5. 11. -7.  3.])


In [ ]:
cp.roots()

array([0.64713013+0.j        , 0.8431016 -1.36552314j,
       0.8431016 +1.36552314j])

## b

A is not diagonalizable

# 7

In [ ]:
lA = [[0, 1], [-8, 4]]
charpoly(lA)

x**2 - 4*x + 8

$A$ is diagonlizable if $A\in \mathbb{C}^{2\times 2}$

## b

In [ ]:
lA = [[1, 1, 1],
     [1, 1, 1], 
     [1, 1, 1]]
charpoly(lA)

x**3 - 3*x**2

In [ ]:
eigen_decomposition(A)

(array([[ 5.77350269e-01, -6.51147040e-17,  6.09781659e-01],
        [ 5.77350269e-01, -7.07106781e-01, -7.75129861e-01],
        [ 5.77350269e-01,  7.07106781e-01,  1.65348202e-01]]),
 array([[ 3.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  6.16297582e-33,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00, -7.50963641e-17]]),
 array([[ 0.57735027,  0.57735027,  0.57735027],
        [-0.72705632, -0.34357862,  1.07063494],
        [ 1.0932875 , -0.54664375, -0.54664375]]))

## C

In [ ]:
lA = [[5, 4, 2, 1],
      [0, 1, -1, -1], 
      [-1, -1, 3, 0], 
      [1, 1, -1, 2]]
charpoly(lA)

x**4 - 11*x**3 + 42*x**2 - 64*x + 32

In [ ]:
cp = np.polynomial.polynomial.Polynomial([32, -64, 42, -11, 1])
cp.roots()

array([1.+0.00000000e+00j, 2.+0.00000000e+00j, 4.-4.77427141e-08j,
       4.+4.77427141e-08j])

$A$ is not diagonlizable in $\mathbb{R}^{n\times n}%

## d

In [ ]:
lA = [[5, -6, -6],
      [-1, 4, 2],
      [3, -6, -4]]
charpoly(lA)

x**3 - 5*x**2 + 8*x - 4

In [ ]:
cp = np.polynomial.polynomial.Polynomial([-4, 8, -5, 1])
cp.roots()

array([1.        , 1.99999996, 2.00000004])

In [ ]:
eig(lA)[0]

array([1., 2., 2.])

In [ ]:
eig(lA)[1]

array([[ 0.6882472 , -0.62406387,  0.9427575 ],
       [-0.22941573,  0.37401343,  0.24308314],
       [ 0.6882472 , -0.68604537,  0.22829561]])

In [ ]:
eigen_decomposition(lA)

(array([[ 0.6882472 , -0.62406387,  0.9427575 ],
        [-0.22941573,  0.37401343,  0.24308314],
        [ 0.6882472 , -0.68604537,  0.22829561]]), array([[1., 0., 0.],
        [0., 2., 0.],
        [0., 0., 2.]]), array([[-4.35889894,  8.71779789,  8.71779789],
        [-3.79749605,  8.50038207,  6.63095674],
        [ 1.72909781, -0.73743109, -1.97490818]]))

# 4.9

In [ ]:
lA = [[3, 2, 2], [2, 3, -2]]
svd(lA)

(array([[-0.70710678, -0.70710678],
        [-0.70710678,  0.70710678]]), array([[5., 0., 0.],
        [0., 3., 0.]]), array([[-7.07106781e-01, -7.07106781e-01, -6.47932334e-17],
        [-2.35702260e-01,  2.35702260e-01, -9.42809042e-01],
        [-6.66666667e-01,  6.66666667e-01,  3.33333333e-01]]))

# 4.9

In [ ]:
lA = [[2, 2],
      [-1, 1]]
svd(lA)

(array([[-1.00000000e+00,  1.11022302e-16],
        [ 1.11022302e-16,  1.00000000e+00]]), array([[2.82842712, 0.        ],
        [0.        , 1.41421356]]), array([[-0.70710678, -0.70710678],
        [-0.70710678,  0.70710678]]))

# 4.10

In [ ]:
lA = [[3, 2, 2],
      [2, 3, -2]]
matrix_approximate(lA, 1)

(2, 2) (2, 1) (1, 3)


array([[2.50000000e+00, 2.50000000e+00, 2.29078674e-16],
       [2.50000000e+00, 2.50000000e+00, 2.29078674e-16]])

# 4.11

$A^TAx = \lambda x$ \\
$AA^T(Ax) = \lambda (Ax)$